In [1]:
'''
Tasks
- Replace gl movieId with tmbd id
- Add new 2023-2025 movies in from Letterboxd with new uniqueIds and tmdb ids for movies
'''

'\nTasks\n- Replace gl movieId with tmbd id\n- Add new 2023-2025 movies in from Letterboxd with new uniqueIds and tmdb ids for movies\n'

In [2]:
import os
import pandas as pd
from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client(project="film-wizard-453315")
print("Authenticated successfully!")

Authenticated successfully!


## Existing GL ratings plus tmdb ids

In [ ]:
# Define batch size
batch_size = 1_000_000  # Adjust as needed
offset = 0
all_data = []

while True:
    # Define query with LIMIT and OFFSET
    sample_query = f"""
    SELECT userId, movieId, rating
    FROM `film-wizard-453315.Grouplens.raw_grouplens_ratings`
    LIMIT {batch_size} OFFSET {offset}
    """
    
    # Fetch batch
    batch_df = client.query(sample_query).to_dataframe()
    
    # Break loop if no more data
    if batch_df.empty:
        print("No more data to fetch. Stopping batch process.")
        break
    
    # Append to list
    all_data.append(batch_df)
    
    # Confirm batch retrieval
    print(f"Batch added: {len(batch_df)} rows (Offset: {offset})")
    
    # Move offset
    offset += batch_size

# Concatenate all batches into final DataFrame
gl_df = pd.concat(all_data, ignore_index=True)

# Display final result
gl_df

/Users/oliverramsaygray/.pyenv/versions/3.10.6/envs/film_wizard/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Batch added: 1000000 rows (Offset: 0)
Batch added: 1000000 rows (Offset: 1000000)
Batch added: 1000000 rows (Offset: 2000000)


In [ ]:
# Fetch links table
sample_query = """
SELECT *
FROM `film-wizard-453315.Grouplens.raw_grouplens_links`
"""
links_df = client.query(sample_query).to_dataframe()
links_df

In [ ]:
initial_gl_tmdb_df = pd.merge(gl_df, links_df, on='movieId', how='inner') #163 rows don't have tmdb ids so are dropped if using 'inner'. This is 28 unique movies.
initial_gl_tmdb_df = initial_gl_tmdb_df[["userId", "movieId", "tmdbId", "rating"]]
initial_gl_tmdb_df

## To check how many unique gl movieIds are being lost with left vs inner join
# nan_movie_ids_count = initial_gl_tmdb_df[initial_gl_tmdb_df['tmdbId'].isna()]['movieId'].nunique()
# nan_movie_ids_count

## Letterboxd ratings plus tmdb ids

In [ ]:
# Fetch Letterboxd reviews
sample_query = """
SELECT *
FROM `film-wizard-453315.Letterboxd.letterboxed_user_reviews`
"""
letterboxd_df = client.query(sample_query).to_dataframe()
letterboxd_df['rating_val'] = letterboxd_df['rating_val']/2
letterboxd_df

In [ ]:
# Fetch tmdb data
sample_query = """
SELECT tmdbId, title, release_date
FROM `film-wizard-453315.tmdb_metadata.all_movies_combined`
"""
tmdb_df = client.query(sample_query).to_dataframe()

print('')

tmdb_df['release_date'] = pd.to_datetime(tmdb_df['release_date'])

# 1) Get the data types of each column
print("Data types of each column:")
print(tmdb_df.dtypes)

print('')

# 2) Count the non-NaN values
release_date_count = tmdb_df['title'].count()
print("Non-NaN values in title column:", release_date_count)

release_date_count = tmdb_df['release_date'].count()
print("Non-NaN values in release_date column:", release_date_count)

tmdb_df

In [ ]:
from rapidfuzz import process, fuzz
import pandas as pd
import re

# Function to clean movie titles for better fuzzy matching
def clean_title(title):
    """
    Cleans a movie title by:
    - Converting to lowercase
    - Replacing hyphens with spaces
    - Removing all characters except letters, numbers, and spaces
    """
    if not isinstance(title, str):  # Handle None values
        return ""
    title = title.lower()
    title = title.replace(" - ", " ")  # Normalize spaces around hyphens
    title = title.replace("-", " ")    # Convert remaining hyphens to spaces
    title = re.sub(r'[^a-z0-9 ]', '', title)  # Remove punctuation
    return title.strip()

# Function to extract the release year from the title
def extract_year(title):
    """
    Extracts a 4-digit year from a movie title, if present.
    Returns the year as an integer or None if no year is found.
    """
    if not isinstance(title, str):  # Handle None values
        return None
    match = re.search(r'\b(19|20)\d{2}\b', title)  # Look for years between 1900-2099
    return int(match.group()) if match else None

# Apply preprocessing to clean movie titles
letterboxd_df['clean_movie_id'] = letterboxd_df['movie_id'].apply(clean_title)
tmdb_df['clean_title'] = tmdb_df['title'].apply(clean_title)

# Extract years from both dataframes
letterboxd_df['year'] = letterboxd_df['movie_id'].apply(extract_year)
tmdb_df['year'] = tmdb_df['title'].apply(extract_year)

# Convert TMDB titles into a list for faster lookup
tmdb_titles = tmdb_df['clean_title'].tolist()

# Lists to store match results
matched_titles = []
fuzzy_scores = []
matched_tmdb_ids = []

# Perform fuzzy matching for each movie in the Letterboxd dataset
for index, row in letterboxd_df.iterrows():
    movie_id = row['clean_movie_id']
    movie_year = row['year']  # Extract year for additional filtering

    if not movie_id:  # Skip empty movie IDs
        matched_titles.append(None)
        fuzzy_scores.append(None)
        matched_tmdb_ids.append(None)
        continue

    # Get top 5 matches using a strict ratio-based scoring system
    matches = process.extract(movie_id, tmdb_titles, scorer=fuzz.ratio, limit=5)

    # Filter to only matches that score above 90 (strong match)
    valid_matches = [match for match in matches if match[1] >= 90]

    # If a release year is available, prefer matches with the same year
    if movie_year:
        year_matched_df = tmdb_df[
            (tmdb_df['clean_title'].isin([match[0] for match in valid_matches])) & 
            (tmdb_df['year'] == movie_year)
        ]
        if not year_matched_df.empty:
            best_match = (year_matched_df.iloc[0]['clean_title'], 100)  # Assign perfect score
        else:
            best_match = valid_matches[0] if valid_matches else matches[0]
    else:
        best_match = valid_matches[0] if valid_matches else matches[0]

    # Retrieve the original TMDB title and ID
    match_row = tmdb_df.loc[tmdb_df['clean_title'] == best_match[0]]
    matched_titles.append(match_row['title'].values[0] if not match_row.empty else None)
    fuzzy_scores.append(best_match[1])
    matched_tmdb_ids.append(match_row['tmdbId'].values[0] if not match_row.empty else None)

# Add match results to DataFrame
letterboxd_df['matched_title'] = matched_titles
letterboxd_df['fuzzy_score'] = fuzzy_scores
letterboxd_df['matched_tmdbId'] = matched_tmdb_ids  # Add tmdbId to final DataFrame

# Display the first 50 results
letterboxd_df.head(50)

In [ ]:
letterboxd_df = letterboxd_df[['user_id', 'matched_tmdbId', 'rating_val']]

In [ ]:
# Example: Get the maximum existing userId from gl_df
max_user_id = gl_df['userId'].max()

# Create a mapping of unique user_id values to new sequential numbers
unique_users = letterboxd_df['user_id'].unique()
user_id_mapping = {user: idx for idx, user in enumerate(unique_users, start=max_user_id + 1)}

# Apply the mapping to the user_id column
letterboxd_df['new_user_id'] = letterboxd_df['user_id'].map(user_id_mapping)

# Display the updated dataframe
letterboxd_df = letterboxd_df[['new_user_id', 'matched_tmdbId', 'rating_val']]
letterboxd_df

In [ ]:
letterboxd_df = letterboxd_df.rename(columns={'new_user_id': 'userId', 
                                      'matched_tmdbId': 'movieId', 
                                      'rating_val': 'rating'})
master_df = pd.concat([gl_df, letterboxd_df], ignore_index=True)
master_df